# This notebook represents how to combine multiple datasets

#  *we want to check if Presidential Voting, Minimum Wage and Uneployement rate (**all datasets**) have any relations with each other*

In [6]:
import pandas as pd
import numpy as np

unemployment_by_county = pd.read_csv("../datasets/unemployment-county-rate2016.csv")

unemployment_by_county.head()

# index by by year


,Year,Month,State,County,Rate
0,2015,February,Mississippi,Newton County,6.1
1,2015,February,Mississippi,Panola County,9.4
2,2015,February,Mississippi,Monroe County,7.9
3,2015,February,Mississippi,Hinds County,6.1
4,2015,February,Mississippi,Kemper County,10.6


In [7]:
df = pd.read_csv("../datasets/minwage.csv")

actual_min_wage = pd.DataFrame()

for name, group in df.groupby("State"):
    if actual_min_wage.empty:
        actual_min_wage = group.set_index("Year")[ ["State.Minimum.Wage.2020.Dollars"] ].rename(
            columns={"State.Minimum.Wage.2020.Dollars" : str(name) + " Min Wage"})
    else:
        actual_min_wage = actual_min_wage.join(
            group.set_index("Year")[ ["State.Minimum.Wage.2020.Dollars"]].rename(
                columns={"State.Minimum.Wage.2020.Dollars" : str(name)}))

actual_min_wage.head() # we get values with 0 (nothing) so we need to replace with NaN and drop
actual_min_wage = actual_min_wage.replace(0, np.NaN).dropna(axis=1)
actual_min_wage.head()

,Alaska,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Guam,Hawaii,Idaho,...,Oregon,Pennsylvania,Rhode Island,South Dakota,Utah,Vermont,Washington,West Virginia,Wisconsin,Wyoming
Year,,,,,,,,,,,,,,,,,,,,,
1968,15.61,1.16,12.26,7.43,10.41,9.29,9.29,9.29,9.29,8.55,...,9.29,8.55,10.41,3.16,7.43,10.41,11.89,7.43,9.29,8.92
1969,14.80,1.10,11.63,7.05,9.87,8.81,8.81,8.81,8.81,8.11,...,8.81,8.11,9.87,3.00,7.05,9.87,11.28,7.05,8.81,8.46
1970,14.00,7.33,11.00,6.67,10.67,8.33,10.67,10.67,10.67,8.33,...,8.33,8.67,10.67,6.67,6.67,10.67,10.67,6.67,8.67,8.67
1971,13.41,7.03,10.54,6.39,10.22,7.98,10.22,10.22,10.22,7.98,...,7.98,8.30,10.22,6.39,6.39,10.22,10.22,6.39,8.30,8.30
1972,12.99,7.43,10.21,6.19,11.45,9.90,9.90,11.76,9.90,8.66,...,7.74,9.90,9.90,6.19,7.43,9.90,9.90,7.43,8.97,9.28


# We want to get the minimum wage of each state and then insert it into the unemployment_by_county dataframe

# But we have a problem: Our Min Wage data is filtered by **State** while our Unemployment By County dataset and Presidential Voting dataset are filtered by **County**

# *but we can still create a workaround for this*

In [8]:
# first lets create a method to get the min wage by Year and State
def get_min_wage(year, state):
    try:
        return actual_min_wage.loc[year][state] # loc uses the labels to access rows or cols
    except:
        return np.NaN
    
get_min_wage(2012, "Colorado")

8.61

In [9]:
# #  map new column called "min_wage" using get_min_wage() function and accessing year and state
# # from unemployment_by_country dataframe
# SLOWEST PART
import time

start = time.time()
unemployment_by_county["min_wage"] = list(map(
    get_min_wage, unemployment_by_county["Year"], unemployment_by_county["State"]))

end = time.time()
print("Time take: " + str(end - start))

Time take: 46.044575929641724


In [10]:
unemployment_by_county.head()
# we can now see below the the new column min-wage has been mapped

,Year,Month,State,County,Rate,min_wage
0,2015,February,Mississippi,Newton County,6.1,NaN
1,2015,February,Mississippi,Panola County,9.4,NaN
2,2015,February,Mississippi,Monroe County,7.9,NaN
3,2015,February,Mississippi,Hinds County,6.1,NaN
4,2015,February,Mississippi,Kemper County,10.6,NaN


Let's check if there is a correlation between Unemployment Rate and Minimum Wage
using PMCC (Pearson/Product Moment Correlation Coefficient)

In [11]:
# unemployment_by_county[["Rate", "min_wage"]].corr(method="pearson") 
unemployment_by_county[["Rate", "min_wage"]].corr(method="pearson")

,Rate,min_wage
Rate,1.000000,0.153035
min_wage,0.153035,1.000000


Similarly, Let's measure covariance (how much they vary) between Unemployment Rate and Minimum Wage
using PMCC (Pearson/Product Moment Correlation Coefficient)

In [12]:
unemployment_by_county[["Rate", "min_wage"]].cov()

,Rate,min_wage
Rate,9.687873,0.673099
min_wage,0.673099,2.000364


In [13]:
unemployment_by_county.dropna(inplace=True)
# print(np.shape(unemployment_by_county[["Rate"]["min_wage"]]))
x = np.array(unemployment_by_county)
print(np.shape(x)) # matrix dimensions

(528158, 6)


### Loading in Presidential Data:

In [14]:
pres2016 = pd.read_csv("../datasets/pres2016_voteresults.csv")
print(pres2016.columns)

pres2016.head()

Index(['county', 'fips', 'cand', 'st', 'pct_report', 'votes', 'total_votes',
       'pct', 'lead'],
      dtype='object')


,county,fips,cand,st,pct_report,votes,total_votes,pct,lead
0,NaN,US,Donald Trump,US,0.9951,60350241.0,127592176.0,0.472993,Donald Trump
1,NaN,US,Hillary Clinton,US,0.9951,60981118.0,127592176.0,0.477938,Donald Trump
2,NaN,US,Gary Johnson,US,0.9951,4164589.0,127592176.0,0.032640,Donald Trump
3,NaN,US,Jill Stein,US,0.9951,1255968.0,127592176.0,0.009844,Donald Trump
4,NaN,US,Evan McMullin,US,0.9951,451636.0,127592176.0,0.003540,Donald Trump


In [15]:
unemployment_by_county.head()

,Year,Month,State,County,Rate,min_wage
82,2015,February,Oklahoma,Major County,2.6,2.18
83,2015,February,Oklahoma,Pottawatomie County,4.5,2.18
84,2015,February,Oklahoma,Johnston County,6.5,2.18
85,2015,February,Oklahoma,Jefferson County,5.0,2.18
86,2015,February,Oklahoma,Beaver County,2.8,2.18


In [16]:
# make a new dataframe based off unemployment_by_country dataframe (for Year 2015 and Month Febuary)
county_2015 = unemployment_by_county.copy()[ # use copy to avoid warnings
    (unemployment_by_county["Year"]==2015) & 
    (unemployment_by_county["Month"]=="February")]


county_2015.head()

,Year,Month,State,County,Rate,min_wage
82,2015,February,Oklahoma,Major County,2.6,2.18
83,2015,February,Oklahoma,Pottawatomie County,4.5,2.18
84,2015,February,Oklahoma,Johnston County,6.5,2.18
85,2015,February,Oklahoma,Jefferson County,5.0,2.18
86,2015,February,Oklahoma,Beaver County,2.8,2.18


## We want to replace the states in county_2015 dataframe with abbreviations ( csv made in the past) so that we can match the state from county_2015 to states in pres2016
##

In [17]:
# states - abbreviations based off postal code
pres2016["st"].unique()

array(['US', 'CA', 'FL', 'TX', 'NY', 'PA', 'IL', 'OH', 'MI', 'NC', 'GA',
       'VA', 'NJ', 'MA', 'WI', 'MN', 'MO', 'WA', 'IN', 'CO', 'TN', 'MD',
       'SC', 'AL', 'AZ', 'LA', 'KY', 'OR', 'CT', 'IA', 'OK', 'MS', 'KS',
       'NV', 'AR', 'UT', 'NE', 'NM', 'ME', 'NH', 'WV', 'ID', 'MT', 'RI',
       'DE', 'HI', 'SD', 'ND', 'VT', 'DC', 'WY', 'AK', nan], dtype=object)

In [18]:
state_abbv = pd.read_csv("../datasets/fixed_state_abbreviations.csv", index_col=0) # set first field as index
state_abbv = state_abbv[["Postal Code"]]
state_abbv.head()

,Postal Code
State Name/District,
Alabama,AL
Alaska,AK
Arizona,AZ
Arkansas,AR
California,CA


In [19]:
# convert to dictionary
state_abbv_dict = state_abbv.to_dict()["Postal Code"]
state_abbv_dict

{'Alabama': 'AL',
 'Alaska': 'AK',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'District of Columbia': 'DC',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 'Virginia': 'VA',
 'Washington': 'WA',
 'West Virginia': 'WV',
 'Wisconsin': 'WI',
 'Wyoming': 'WY

In [20]:
# map the State names from county_state dataframe to abbreviations (replacing the state names with state abbreviations)
county_2015["State"] = county_2015["State"].map(state_abbv_dict)

In [21]:
county_2015.tail()

,Year,Month,State,County,Rate,min_wage
2797,2015,February,ME,Somerset County,8.4,8.18
2798,2015,February,ME,Oxford County,6.8,8.18
2799,2015,February,ME,Knox County,6.1,8.18
2800,2015,February,ME,Piscataquis County,7.0,8.18
2801,2015,February,ME,Aroostook County,7.2,8.18


In [22]:
print(len(county_2015))
print(len(pres2016))

1569
18475


In [23]:
pres2016.columns

Index(['county', 'fips', 'cand', 'st', 'pct_report', 'votes', 'total_votes',
       'pct', 'lead'],
      dtype='object')

In [24]:
pres2016.rename(columns={"county": "County", "st": "State"}, inplace=True)
pres2016.head()

,County,fips,cand,State,pct_report,votes,total_votes,pct,lead
0,NaN,US,Donald Trump,US,0.9951,60350241.0,127592176.0,0.472993,Donald Trump
1,NaN,US,Hillary Clinton,US,0.9951,60981118.0,127592176.0,0.477938,Donald Trump
2,NaN,US,Gary Johnson,US,0.9951,4164589.0,127592176.0,0.032640,Donald Trump
3,NaN,US,Jill Stein,US,0.9951,1255968.0,127592176.0,0.009844,Donald Trump
4,NaN,US,Evan McMullin,US,0.9951,451636.0,127592176.0,0.003540,Donald Trump


In [25]:
for df in [county_2015, pres2016]:
    df.set_index(["County", "State"], inplace=True)

In [26]:
pres2016.head()

fips             cand  pct_report       votes  total_votes  \
County State                                                              
NaN    US      US     Donald Trump      0.9951  60350241.0  127592176.0   
       US      US  Hillary Clinton      0.9951  60981118.0  127592176.0   
       US      US     Gary Johnson      0.9951   4164589.0  127592176.0   
       US      US       Jill Stein      0.9951   1255968.0  127592176.0   
       US      US    Evan McMullin      0.9951    451636.0  127592176.0   

                   pct          lead  
County State                          
NaN    US     0.472993  Donald Trump  
       US     0.477938  Donald Trump  
       US     0.032640  Donald Trump  
       US     0.009844  Donald Trump  
       US     0.003540  Donald Trump

In [27]:
# Lets use only voting data for Donald Trump as the dataset is big
pres2016 = pres2016[pres2016["cand"] == "Donald Trump"]
# create a new datafram where the candidate is "Donald Trump"  and pct (percentage votes)
pres2016 = pres2016[["pct"]]
pres2016.dropna(inplace=True)
pres2016.head()

pct
County State          
NaN    US     0.472993
       CA     0.330641
       FL     0.490640
       TX     0.525830
       NY     0.374752

## Now we can merge county_2015 and pres2016 since they both have the same columns "County" & "State"

In [28]:
# merge both dataframes into a new datafram
merged_dataframes = county_2015.merge(pres2016, on=["County", "State"])
merged_dataframes.dropna(inplace=True)

In [29]:
merged_dataframes.head()
# as we can see below we have merged the 2 dataframes

,,Year,Month,Rate,min_wage,pct
County,State,,,,,
Major County,OK,2015,February,2.6,2.18,0.864960
Pottawatomie County,OK,2015,February,4.5,2.18,0.701342
Johnston County,OK,2015,February,6.5,2.18,0.770057
Jefferson County,OK,2015,February,5.0,2.18,0.812367
Beaver County,OK,2015,February,2.8,2.18,0.888243


In [30]:
merged_dataframes.drop("Year", axis=1, inplace=True)
merged_dataframes.head()

,,Month,Rate,min_wage,pct
County,State,,,,
Major County,OK,February,2.6,2.18,0.864960
Pottawatomie County,OK,February,4.5,2.18,0.701342
Johnston County,OK,February,6.5,2.18,0.770057
Jefferson County,OK,February,5.0,2.18,0.812367
Beaver County,OK,February,2.8,2.18,0.888243


In [31]:
merged_dataframes.corr()

# below shows the following: 
# Unemployment Rate & Minimum Wage -> weak positive correlation, as unemployment rate increases, minimum wage increases (but note that 0.18 is very close to 0 - no correlation)
# Unemployment Rate and Percentage of Votes (Trump) -> very weak negative correlation -> As votes went up, unemployment went down
# Minimum Wage and Percentage of Votes (Trump) -> negative correlation -> As Votes went up, min wage went down

,Rate,min_wage,pct
Rate,1.000000,0.186404,-0.085985
min_wage,0.186404,1.000000,-0.324942
pct,-0.085985,-0.324942,1.000000


In [32]:
merged_dataframes.cov() # quite alot of variance which MEANS THERES MORE FACTORS IN PLAY FOR POLITICS

,Rate,min_wage,pct
Rate,5.743199,0.705474,-0.031771
min_wage,0.705474,2.494004,-0.079120
pct,-0.031771,-0.079120,0.023772
